# **FIAP Pós-Tech - IA para Devs**

## Grupo 14 (1IADT) - Alunos

* Marcelo Henriques da Fonseca - RM353865
* Marcos Lopes da Silva Junior - RM353763
* Ricardo Báfica Pontes - RM353866

# **Descrição do projeto**

## O problema

No Tech Challenge desta fase, você precisa executar o fine-tuning de um
foundation model (Llama, BERT, MISTRAL etc.), utilizando o dataset "The
AmazonTitles-1.3MM". O modelo treinado deverá:
- Receber perguntas com um contexto obtido por meio do arquivo json
“trn.json” que está contido dentro do dataset.
- A partir do prompt formado pela pergunta do usuário sobre o título do
produto, o modelo deverá gerar uma resposta baseada na pergunta do
usuário trazendo como resultado do aprendizado do fine-tuning os
dados da sua descrição.

## Fluxo de Trabalho Atualizado

**1. Escolha do Dataset:**


**Descrição:** o The AmazonTitles-1.3MM consiste em consultas textuais
reais de usuários e títulos associados de produtos relevantes encontrados na
Amazon e suas descrições, medidos por ações implícitas ou explícitas dos
usuários.

**2. Preparação do Dataset:**

Faça o download do dataset AmazonTitles-1.3MM e utilize o arquivo
“trn.json”. Nele, você utilizará as colunas “title” e “content”, que contêm título e
descrição respectivamente. Prepare os prompts para o fine-tuning garantindo
que estejam organizados de maneira adequada para o treinamento do modelo escolhido. Limpe e pré-processe os dados conforme necessário para o
modelo escolhido.


**3. Chamada do Foundation Model**

Importe o foundation model que será utilizado e faça um teste apresentando o resultado atual do modelo antes do treinamento (para que se obtenha uma base de análise após o fine-tuning), e então será possível avaliar a diferença do resultado gerado.


**4. Execução do Fine-Tuning:**

Execute o fine-tuning do foundation model selecionado (por exemplo, BERT, GPT, Llama) utilizando o dataset preparado. Documente o processo de fine-tuning, incluindo os parâmetros utilizados e qualquer ajuste específico realizado no modelo.


**5. Geração de Respostas:**

Configure o modelo treinado para receber perguntas dos usuários. O modelo deverá gerar uma resposta baseada na pergunta do usuário e nos dados provenientes do fine-tuning, incluindo as fontes fornecidas.


## Entregável

- Documento detalhando o processo de seleção e preparação do dataset.
- Descrição do processo de fine-tuning do modelo, com detalhes dos parâmetros e ajustes utilizados. Código-fonte do processo de finetuning.
- Um vídeo demonstrando o modelo treinado gerando respostas a partir de perguntas do usuário e utilizando o contexto obtido por meio treinamento com o fine-tuning.

# **Projeto**

## Carregando o arquivo JSON

In [26]:
# imports
import pandas as pd
import numpy as np
import json
import html
from google.colab import drive

Primeiro vamos carregar o arquivo JSON que será utilizado no nosso fine-tuning.

Vamos utilizar algumas estratégias para agilizar a leitura do arquivo:

- Ler e processar o arquivo em blocos (chunks), ao invés de linha-a-linha, reduz o overhead de leitura frequente de disco.
- Adicionar linha-a-linha no DataFrame pode ser uma operação custosa. Em vez disso, podemos amazenar os dados em uma lista temporária e criar o DataFrame com uma única operação.

Também podemos utilizar o "html.unescape(xxx)" para tratar os caracteres especias e deixar nossos dados mais legíveis.

In [27]:
# Função para processar e limpar um bloco de dados
def processar_bloco(bloco):
    dados = []
    for line in bloco:
        try:
            json_data = json.loads(line)
            title = html.unescape(json_data['title'])
            content = html.unescape(json_data['content'])
            dados.append([title, content])
        except json.JSONDecodeError:
            continue
    return pd.DataFrame(dados, columns=['title', 'content'])

In [28]:
# Função para carregar o JSON em blocos e processar
def carregar_json(arquivo_json, tamanho_bloco):
    df_final = pd.DataFrame(columns=['title', 'content'])
    bloco = []

    with open(arquivo_json, 'r', newline='\n') as f:
        for line in f:
            bloco.append(line)

            if len(bloco) >= tamanho_bloco:
                df_bloco = processar_bloco(bloco)
                df_final = pd.concat([df_final, df_bloco], ignore_index=True)
                bloco = []

        if bloco:
            df_bloco = processar_bloco(bloco)
            df_final = pd.concat([df_final, df_bloco], ignore_index=True)

    return df_final, len(df_final)

Neste caso, vamos optar por usar o arquivo direto do Google Drive.

In [29]:
# Tamanho do bloco
tamanho_bloco = 50000

# Montar o Google Drive
drive.mount('/content/drive')

# Caminho para o arquivo JSON no Google Drive
arquivo_json = '/content/drive/My Drive/FIAP/IA_para_Devs/Tech_Challenge_3/trn.json'

# Chamando a função otimizada
df, qtd_linhas = carregar_json(arquivo_json, tamanho_bloco)

# Saída de verificação
print(f'Linhas processadas: {qtd_linhas}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Linhas processadas: 2248619


## Analisando e tratando os dados

In [30]:
# Comando para verificar as primeiras linhas do DataFrame
df.head()

,title,content
0,Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...
1,Adult Ballet Tutu Yellow,
2,The Way Things Work: An Illustrated Encycloped...,
3,Mog's Kittens,Judith Kerr’s best–selling adventures of that ...
4,Misty of Chincoteague,


In [31]:
df.shape

(2248619, 2)

Podemos verificar que esse DataFrame possui mais de 2 milhões de linhas de dados.

Conforme o escopo do projeto, carregamos apenas os dados de "title" e "content".

"title": título do produto

"content": descrição do produto

Podemos verificar logo de início que existem linhas do DataFrame que possuem um "title" mas nenhum "content". Isso precisa ser tratado.

In [32]:
def contar_vazios_ou_nulos(coluna):
    nulos = coluna.isnull().sum()
    vazios = (coluna == '').sum()
    return nulos + vazios

In [33]:
nulos_vazios_title = contar_vazios_ou_nulos(df['title'])
nulos_vazios_content = contar_vazios_ou_nulos(df['content'])

print(f"Valores nulos ou vazios em 'title': {nulos_vazios_title}")
print(f"Valores nulos ou vazios em 'content': {nulos_vazios_content}")

Valores nulos ou vazios em 'title': 126834
Valores nulos ou vazios em 'content': 749901


Com os prints acima, podemos verificar que existem vários campos de "title" e "content" vazios. Essas linhas não nos servirão e precisam ser excluídas.

In [34]:
# Excluir registros onde qualquer uma das colunas 'title' ou 'content' contenha string nula ou vazia
df = df[(df['title'].str.strip() != '') & (df['content'].str.strip() != '')]

In [35]:
df.head()

,title,content
0,Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...
3,Mog's Kittens,Judith Kerr’s best–selling adventures of that ...
7,Girls Ballet Tutu Neon Blue,Dance tutu for girls ages 2-8 years. Perfect f...
12,The Prophet,"In a distant, timeless place, a mysterious pro..."
13,Rightly Dividing the Word,--This text refers to thePaperbackedition.


In [36]:
df.shape

(1390100, 2)

Após essa limpeza, o número de registros cai para um pouco melhor de 1.5 milhões.

Vamos checar também se existem dados duplicados no campo "title" e, se necessário, excluí-los.

Seria problemático para o nosso modelo seria se houvessem produtos com nomes iguais mas descrições diferentes.

In [37]:
# Contar duplicatas no título
duplicates_title = df.duplicated(subset='title').sum()

print(f"Número de duplicatas baseadas em 'title': {duplicates_title}")

# Remover duplicatas baseadas no título
df = df.drop_duplicates(subset='title')

Número de duplicatas baseadas no 'title': 39961


In [38]:
df.shape

(1350139, 2)

Abaixo, vamos deixar os dados um pouco mais consistentes removendo espaços em branco no começo e no final das strings.

In [39]:
# Remover espaços em branco no início e no final
df['title'] = df['title'].str.strip()
df['content'] = df['content'].str.strip()

In [40]:
df.head()

,title,content
0,Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...
3,Mog's Kittens,Judith Kerr’s best–selling adventures of that ...
7,Girls Ballet Tutu Neon Blue,Dance tutu for girls ages 2-8 years. Perfect f...
12,The Prophet,"In a distant, timeless place, a mysterious pro..."
13,Rightly Dividing the Word,--This text refers to thePaperbackedition.


## Dividindo os dados

Nosso dataset é grande demais para ser treino de uma vez só.

Vamos optar por dividi-lo em partes e treino-lo separadamente nos próximos notebooks.

No bloco abaixo nós definimos um número de linhas por arquivo e chamamos uma função que irá dividir o DataFrame e salvo-lo em partes em uma pasta do Google Drive.

In [41]:
# Função para dividir o DataFrame e salvar em arquivos JSON
def dividir_e_salvar_em_json(df, linhas_por_arquivo, pasta_destino):
    # Total de partes
    total_partes = (len(df) // linhas_por_arquivo) + 1

    for i in range(total_partes):
        # Definir o intervalo para cada pedaço
        inicio = i * linhas_por_arquivo
        fim = (i + 1) * linhas_por_arquivo

        # Extrair a parte do DataFrame
        df_pedaco = df.iloc[inicio:fim]

        # Definir o nome do arquivo JSON (exemplo: trn_100k_1.json, trn_100k_2.json, etc.)
        nome_arquivo = f'{pasta_destino}trn_100k_{i+1}.json'

        # Converter a parte do DataFrame para JSON e salvar
        df_pedaco.to_json(nome_arquivo, orient='records', lines=True, force_ascii=False)

        print(f'Arquivo {nome_arquivo} salvo com sucesso!')

In [42]:
# Número máximo de linhas por arquivo
linhas_por_arquivo = 100000

pasta_destino = '/content/drive/My Drive/FIAP/IA_para_Devs/Tech_Challenge_3/'

# Chamar a função para dividir e salvar
dividir_e_salvar_em_json(df, linhas_por_arquivo, pasta_destino)

Arquivo /content/drive/My Drive/FIAP/IA_para_Devs/Tech_Challenge_3/trn_100k_1.json salvo com sucesso!
Arquivo /content/drive/My Drive/FIAP/IA_para_Devs/Tech_Challenge_3/trn_100k_2.json salvo com sucesso!
Arquivo /content/drive/My Drive/FIAP/IA_para_Devs/Tech_Challenge_3/trn_100k_3.json salvo com sucesso!
Arquivo /content/drive/My Drive/FIAP/IA_para_Devs/Tech_Challenge_3/trn_100k_4.json salvo com sucesso!
Arquivo /content/drive/My Drive/FIAP/IA_para_Devs/Tech_Challenge_3/trn_100k_5.json salvo com sucesso!
Arquivo /content/drive/My Drive/FIAP/IA_para_Devs/Tech_Challenge_3/trn_100k_6.json salvo com sucesso!
Arquivo /content/drive/My Drive/FIAP/IA_para_Devs/Tech_Challenge_3/trn_100k_7.json salvo com sucesso!
Arquivo /content/drive/My Drive/FIAP/IA_para_Devs/Tech_Challenge_3/trn_100k_8.json salvo com sucesso!
Arquivo /content/drive/My Drive/FIAP/IA_para_Devs/Tech_Challenge_3/trn_100k_9.json salvo com sucesso!
Arquivo /content/drive/My Drive/FIAP/IA_para_Devs/Tech_Challenge_3/trn_100k_10.jso